In [8]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers
from keras.preprocessing import image
from skimage.io import imread
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import SGD
from sklearn.metrics import classification_report,confusion_matrix
import cv2 as cv

In [12]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow as tf
from tensorflow.keras.applications import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.losses import *
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.preprocessing.image import *
from tensorflow.keras.utils import *
from sklearn.metrics import *
from sklearn.model_selection import *
import tensorflow.keras.backend as K
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob
from skimage.io import *
%config Completer.use_jedi = False
import warnings
warnings.filterwarnings('ignore')
print("All modules have been imported")

All modules have been imported


In [10]:
def load_dataset(container_path):
    image_dir = Path(container_path)
    folders = [directory for directory in image_dir.iterdir() if directory.is_dir()]
    categories = [fo.name for fo in folders]

    descr = "A image classification dataset"
    images = []
    flat_data = []
    target = []
    count = 0
    train_img = []
    for i, direc in enumerate(folders):
        for file in direc.iterdir():
            count += 1
            img = imread(file)
            img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
            img_pred = image.img_to_array(img)
            img_pred = img_pred / 255
            train_img.append(img_pred)
    X = np.array(train_img)
    print("count: "+ str(count))
    return X

X = []
X = load_dataset(r"C:\Users\karth\Sunayana\Machine Learning\Diabetic Retinopathy\data\KNN")

y0 = np.zeros(3670)
y1 = np.ones(2128)
y = []
y = np.concatenate((y0,y1), axis=0)
y = []
y = np.concatenate((y0,y1), axis=0)
x_train, x_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
# y_train = np.asarray(y_train).astype('float32').reshape((-1,1))
# y_test = np.asarray(y_test).astype('float32').reshape((-1,1))
print("x_train: "+str(x_train.shape))
print("x_test: "+str(x_test.shape))
print("y_train: "+str(y_train.shape))
print("y_test: "+str(y_test.shape))

count: 5798
x_train: (4638, 224, 224, 1)
x_test: (1160, 224, 224, 1)
y_train: (4638, 2)
y_test: (1160, 2)


# ResNet50 (7 Dense Layers)

In [16]:
model = Sequential()
model.add(ResNet50(input_shape=(224,224,1),include_top=False,weights=None))
for layer in model.layers:
    layer.trainable = False
model.add(Flatten())
model.add(Dense(16,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(units= 2, activation='softmax'))

c3=tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.1,
    patience=2,
    mode="auto",
    min_delta=0.001,
    cooldown=0,
    min_lr=0.01
)
model.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy','AUC'])
history=model.fit(x_train,y_train,epochs=5,batch_size=5,validation_split=0.2)



Epoch 1/5
742/742 [==============================] - 199s 265ms/step - loss: 0.7334 - accuracy: 0.5784 - auc: 0.5839 - val_loss: 0.7278 - val_accuracy: 0.6228 - val_auc: 0.6227
Epoch 2/5
742/742 [==============================] - 204s 275ms/step - loss: 0.6742 - accuracy: 0.6146 - auc: 0.6322 - val_loss: 0.6627 - val_accuracy: 0.6228 - val_auc: 0.6381
Epoch 3/5
742/742 [==============================] - 297s 400ms/step - loss: 0.6654 - accuracy: 0.6229 - auc: 0.6336 - val_loss: 0.6625 - val_accuracy: 0.6228 - val_auc: 0.6340
Epoch 4/5
742/742 [==============================] - 312s 420ms/step - loss: 0.6606 - accuracy: 0.6318 - auc: 0.6387 - val_loss: 0.6647 - val_accuracy: 0.6228 - val_auc: 0.6203
Epoch 5/5
742/742 [==============================] - 317s 428ms/step - loss: 0.6604 - accuracy: 0.6321 - auc: 0.6320 - val_loss: 0.6612 - val_accuracy: 0.6228 - val_auc: 0.6366


In [20]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23581440  
                                                                 
 flatten_3 (Flatten)         (None, 100352)            0         
                                                                 
 dense_6 (Dense)             (None, 16)                1605648   
                                                                 
 batch_normalization_3 (Batc  (None, 16)               64        
 hNormalization)                                                 
                                                                 
 activation_3 (Activation)   (None, 16)                0         
                                                                 
 dropout_3 (Dropout)         (None, 16)                0         
                                                      

In [75]:
print("Performance Report:")
y_pred6=np.argmax(model.predict(x_test),axis=-1)
Y_test=to_categorical(y_test,5)
y_pred_prb6=model.predict(x_test)
#target=['0','1']
from sklearn import metrics
print('Accuracy score is :', metrics.accuracy_score(y_test, y_pred6))
print('Precision score is :', metrics.precision_score(y_test, y_pred6, average='weighted'))
print('Recall score is :',metrics.recall_score(y_test,y_pred6, average='weighted'))
print('F1 Score is :', metrics.f1_score(y_test, y_pred6,average='weighted'))
print('Cohen Kappa Score:', metrics.cohen_kappa_score(y_test, y_pred6))

Performance Report:
Accuracy score is : 0.74
Precision score is : 0.5476
Recall score is : 0.74
F1 Score is : 0.6294252873563217
Cohen Kappa Score: 0.0
